In [ ]:
from skimage import color
from sklearn import metrics
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2

In [ ]:
train_images = []
train_labels = []
shape = (300, 200)

In [ ]:
train_path = (
    "/home/amirshnll/Projects/pythonProject/new_banknote_model/new_mony_categorical"
)

In [ ]:
for filename in os.listdir(
    "/home/amirshnll/Projects/pythonProject/new_banknote_model/new_mony_categorical"
):
    img = cv2.imread(os.path.join(train_path, filename))

    # Splitting  file names and storing the labels for image in list
    train_labels.append("-".join(filename.split("_")[1:4]))
    # Resize all images to a specific shape
    img = cv2.resize(img, shape)
    RGB = (
        np.mean(cv2.split(img)[0]),
        np.mean(cv2.split(img)[1]),
        np.mean(cv2.split(img)[2]),
    )

    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    histogram_features = hist.flatten()

    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hue_features = cv2.HuMoments(cv2.moments(grayImg)).flatten()

    image_hsv = color.rgb2hsv(img)
    HSV = (
        np.mean(cv2.split(image_hsv)[0]),
        np.mean(cv2.split(image_hsv)[1]),
        np.mean(cv2.split(image_hsv)[2]),
    )

    image_lab = color.rgb2lab(img)
    LAB = (
        np.mean(cv2.split(image_lab)[0]),
        np.mean(cv2.split(image_lab)[1]),
        np.mean(cv2.split(image_lab)[2]),
    )

    train_images.append(np.hstack([hue_features, histogram_features, RGB, HSV, LAB]))

In [ ]:
df = pd.DataFrame(train_labels)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

LE = LabelEncoder()
y = LE.fit_transform(df)

In [ ]:
for channel, col in enumerate(("r", "g", "b")):
    hist = cv2.calcHist([img], [channel], None, [1000], [0, 256])
    plt.plot(hist, color=col)

In [ ]:
mms = MinMaxScaler()
train_images = mms.fit_transform(train_images)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rfc_clf = RandomForestClassifier(n_estimators=500)

with open("test_train_test_split.txt", "a") as file:
    for i in range(11, 31):
        x_train, x_val, y_train, y_val = train_test_split(
            train_images, y, random_state=1, test_size=(i / 100)
        )
        rfc_clf.fit(x_train, y_train)
        p = rfc_clf.predict(x_val)
        e = y_val
        file.write("This accuracy is for {0} test size \n".format(i / 100))
        file.write(metrics.classification_report(e, p))
        file.write("\n -----------------------------------\n")

In [ ]:
from sklearn.model_selection import train_test_split

train_images = np.array(train_images)
x_train, x_val, y_train, y_val = train_test_split(
    train_images, y, random_state=1, test_size=0.15
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

with open("RandomForestClassifier.txt", "a") as file:
    for i in range(100, 3000):
        rfc_clf = RandomForestClassifier(n_estimators=i)
        rfc_clf.fit(x_train, y_train)
        p = rfc_clf.predict(x_val)
        e = y_val
        file.write("This accuracy is for {0} n_estimators \n".format(i))
        file.write(metrics.classification_report(e, p))
        file.write("\n -----------------------------------\n")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc_clf = RandomForestClassifier(n_estimators=400)
rfc_clf.fit(x_train, y_train)

In [ ]:
p = rfc_clf.predict(x_val)
e = y_val

In [ ]:
print(metrics.classification_report(e, p))

In [ ]:
print(metrics.confusion_matrix(e, p))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(n_estimators=1000)
ada_clf.fit(x_train, y_train)

In [ ]:
p = ada_clf.predict(x_val)
e = y_val

In [ ]:
print(metrics.classification_report(e, p))

In [ ]:
print(metrics.confusion_matrix(e, p))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(x_train, y_train)

In [ ]:
p = dtree.predict(x_val)
e = y_val

In [ ]:
print(metrics.classification_report(e, p))

In [ ]:
from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [ ]:
dot_data = StringIO()
export_graphviz(
    rfc_clf, out_file=dot_data, filled=True, rounded=True, special_characters=True
)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
import six
import sys

sys.modules["sklearn.externals.six"] = six
from id3 import Id3Estimator

In [ ]:
classifier = Id3Estimator()
classifier.fit(x_train, y_train)

In [ ]:
p = classifier.predict(x_val)
e = y_val

In [ ]:
print(metrics.classification_report(e, p))

In [ ]:
from IPython.display import Image
from id3 import export_graphviz
import pydot

graph = export_graphviz(classifier.tree_, "tree1.dot", range(x_train.shape[1]))

(graph,) = pydot.graph_from_dot_file("tree1.dot")
graph.write_png("tree1.png")
Image(filename="tree1.png")